# How To Price Call/Put Options in Python Using Real World Data

The Black-Scholes Merton Model (BSM) is a mathematical model used to value option contracts, based on European-style options. It assumes a normal stock price distribution, no transaction costs, no cash flow, no arbitrage opportunities, and constant risk-free rate. The model suggests creating a risk-free portfolio with both stocks and options, requiring continuous rebalancing.

𝐶=𝑆𝑡𝑁(𝑑1)−𝐾𝑒−𝑟𝑡𝑁(𝑑2)
Where d1 and d2 are calculated below:

𝑑1=ln𝑆𝑡/𝐾+(𝑟+𝜎2𝑢2)/𝑡𝜎𝑠𝑡√
and:

𝑑2=𝑑1−𝜎𝑠𝑡√
Where:
𝐶=Call option price
𝑆= Current stock (or other underlying) price
𝐾=Strike price
𝑟=Risk-free interest rate
𝑡=time to maturity 
𝑁=A normal distribution

Today's notebook outlines a process to calculate the call price of an option using the Black-Scholes model parameters, utilizing Python libraries Yfinance and Quandl, and importing necessary libraries.

In [ ]:
import yfinance as yf
from datetime import datetime
import stockquotes
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
def fetch_options_data(ticker_symbol):
    ticker = yf.Ticker(ticker_symbol)
    options_dates = ticker.options
    # We'll use the nearest expiry date for our analysis
    options_data = ticker.option_chain(options_dates[1])
    return options_data.calls, options_data.puts

# Example usage:
jpm_calls, jpm_puts = fetch_options_data('JPM')

In [ ]:
jpm_calls

In [ ]:
## Downloading historical price data for JP Morgan
jpm_stock_data = yf.download("JPM", start="2006-01-01", end="2024-02-24")

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(jpm_stock_data['Close'])
plt.title('JPM Historical Stock Price')
plt.xlabel('Date')
plt.ylabel('Stock Price (USD)')
plt.grid(True)

# The plot shows that the volatility of JP-Morgan stock is higher, So lets calculate the Historical Volatility for JP-Morgan Stock

In [ ]:
def calculate_historical_volatility(stock_data, window=252):
    log_returns = np.log(stock_data['Close'] / stock_data['Close'].shift(1))
    volatility = np.sqrt(window) * log_returns.std()
    return volatility

jpm_volatility = calculate_historical_volatility(jpm_stock_data)
print(f"JPM Historical Volatility: {jpm_volatility}")

In [ ]:
import scipy.stats as si

class BlackScholesModel:
    def __init__(self, S, K, T, r, sigma):
        self.S = S        # Underlying asset price
        self.K = K        # Option strike price
        self.T = T        # Time to expiration in years
        self.r = r        # Risk-free interest rate
        self.sigma = sigma  # Volatility of the underlying asset

    def d1(self):
        return (np.log(self.S / self.K) + (self.r + 0.5 * self.sigma ** 2) * self.T) / (self.sigma * np.sqrt(self.T))
    
    def d2(self):
        return self.d1() - self.sigma * np.sqrt(self.T)
    
    def call_option_price(self):
        return (self.S * si.norm.cdf(self.d1(), 0.0, 1.0) - self.K * np.exp(-self.r * self.T) * si.norm.cdf(self.d2(), 0.0, 1.0))
    
    def put_option_price(self):
        return (self.K * np.exp(-self.r * self.T) * si.norm.cdf(-self.d2(), 0.0, 1.0) - self.S * si.norm.cdf(-self.d1(), 0.0, 1.0))

# Example usage:
bsm = BlackScholesModel(S=120, K=100, T=1, r=0.05, sigma=0.38)
print(f"Call Option Price: {bsm.call_option_price()}")
print(f"Put Option Price: {bsm.put_option_price()}")

# Conclusion 

We have the Call and Put prices given by BSM Model using S=120, K=100, T=1, r=0.05, sigma=0.38(Historical Volatility 
Call Option Price: 31.53394265336798
Put Option Price: 6.656885103439382